In [92]:
import pandas as pd
import ast
import os

In [93]:
categories_key = {
    1: "Gewerkschaftszugehörigkeit",
    2: "rassische und ethnische Herkunft",
    3: "Gesundheitsdaten",
    4: "Sexualleben/sexuelle Orientierung",
    5: "religiöse oder weltanschauliche Überzeugungen",
    6: "Kinder",
    7: "Finanzstatus",
    8: "persönliche Schwächen",
    9: "politische Meinungen",
}

In [111]:
df = pd.read_csv("sensitive_eu_segments_hand_filtered.csv")

# OPTIONAL: filter dataframe to only include sensitive data according to article 9 
# df = df[df.apply(lambda x: x["main_category"] not in (6,7,8), axis=1)]


# Join hand-filtered segments with original dataset

In [112]:
df_orig = pd.read_csv("xandr_segments.csv")
df_orig = df_orig.rename(columns={"Segment Name": "segment_name", "Data Provider Name": "provider_name","Data Provider ID": "provider_id","Segment ID" : "id"})
df_orig = df_orig.set_index("id")
df = df.set_index("id")

In [114]:
df_full = pd.concat([df_orig, df], axis=1, join="inner")
df_full = df_full.rename(columns={"name": "name_processed", "main_category": "category", "id": "segment_id"})
df_full = df_full.sort_values(by=["category", "country", "name_processed"])

cols = ['category', 'country', 'segment_name', 'hit',
        'provider_name', 'provider_id', 'name_processed']
df_full = df_full[cols]

df_full.to_csv("sensitive_eu_segments_master.csv")

# Category frequency

In [115]:
df_full["category"].value_counts()

category
Finanzstatus                                     988
Kinder                                           826
politische Meinungen                              87
Gesundheitsdaten                                  83
religiöse oder weltanschauliche Überzeugungen     35
Sexualleben/sexuelle Orientierung                 13
persönliche Schwächen                             13
Name: count, dtype: int64

# Country frequency

In [116]:
df_full["country"].value_counts()

country
netherlands    377
germany        359
spain          337
france         324
sweden         187
italy          150
denmark        147
finland        107
belgium         12
greece          10
romania         10
austria          8
poland           7
portugal         5
croatia          2
hungary          2
slovakia         1
Name: count, dtype: int64

# Data provider frequency

In [117]:
df_full.value_counts("provider_name")

provider_name
Audiences by Oracle (BlueKai, Datalogix, AddThis)    438
Eyeota                                               324
GroupM NL - GH 6924                                  235
ZeoTap                                               221
Adsquare (Data Provider)                             202
Grapeshot                                            145
Lotame                                               124
Greenhouse Group B.V.                                 78
digitalAudience                                       70
Nielsen Marketing Cloud                               64
KBM Group                                             58
Audiens S.R.L.                                        19
Semasio GmbH (Data Provider)                          19
The Weather Channel, LLC                               9
The ADEX GmbH                                          8
LiveRamp Data Store                                    7
Tru Optik (CTV)                                        7
Weborama SA (Data

In [101]:
dirname_country = "eu_segments_by_country"

df_grouped_country = df_full.groupby("country")

for name, data in df_grouped_country.__iter__():
    data.to_csv(os.path.join(dirname_country, f"{name}.csv"))

In [118]:
dirname_broker = "eu_segments_by_broker"

len_threshold = 10

df_grouped_broker = df_full.groupby("provider_name")

df_other = pd.DataFrame(columns=df_full.columns)

for name, data in df_grouped_broker.__iter__():
    if len(data) < len_threshold:
        df_other = pd.concat([df_other, data])
    else:
        name = name.replace(",", "").replace(".", "").replace(" ", "_")
        data[["provider_name", "provider_id", "segment_name", "country"]].to_csv(os.path.join(dirname_broker, f"{name}.csv"))

df_other.index = df_other.index.set_names("id")
df_other.to_csv(os.path.join(dirname_broker, "other.csv"))

In [109]:
eu_countries = pd.read_json("eu_countries.json")
eu_countries = eu_countries.set_index("name")
eu_countries["hits"] = 0
eu_countries["segments"] = ""

In [150]:
country_list = set(df["country"].values)

highlights = pd.read_csv("segments_for_map_highlighting.csv")

for category, data in df.groupby("category").__iter__():
    foo = eu_countries[["hits", "segments"]].reset_index()
    for i, v in data["country"].value_counts().items():
        foo.loc[foo["name"] == i, "hits"] = v
        foo.loc[foo["name"] == i, "segments"] = "\n".join(highlights[(highlights["category"] == category) & (highlights["country"] == i)]["name"].values)
    category = category.replace("/", "_").replace(" ", "_")
    foo.to_csv(os.path.join("datawrapper_files", f"{category}.csv"))

In [103]:
eu_countries.to_csv("categories_by_country.csv")